In [49]:
#vårt projekt

from typing import Tuple

from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
from torch.utils.data import dataset
from torch.nn.utils.rnn import pad_sequence

import random
import numpy as np
import math

import matplotlib.pyplot as plt
from tqdm import tqdm

In [50]:
#pip install sentencepiece

In [51]:
import sentencepiece as spm

In [52]:
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)

In [53]:
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
DEVICE = torch.device(device)
DEVICE


device(type='mps')

In [54]:
#Alice paths
data_path = '/Users/alicetottie/Downloads/projekt_data/train_paracrawl.en'
data_path_2 = '/Users/alicetottie/Downloads/projekt_data/train_paracrawl.sv'

#Maja paths
# data_path = '/Users/majarygard/Documents/LTH/Projekt i Data/train_paracrawl.en'
# data_path_2 = '/Users/majarygard/Documents/LTH/Projekt i Data/train_paracrawl.sv'

#Pernilla paths
#data_path = 'C:\\Users\\nilla\\Plugg\\project-data\\train_paracrawl.en\\train_paracrawl.en'
#data_path_2 = 'C:\\Users\\nilla\\Plugg\\project-data\\train_paracrawl.sv\\train_paracrawl.sv'


In [55]:
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

with open(data_path_2, "r", encoding="utf-8") as f:
    lines2 = f.read().split("\n")    

In [56]:
len(lines)



4960283

In [57]:
len(lines2)

4960283

In [58]:
num_samples = 1000
input_texts = []
target_texts = []

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text= line.split("\t")
    input_texts.append(input_text)

with open("input_texts.txt", "w") as f:
    for input_text in input_texts:
        f.write('\t'.join(input_text) + '\n')
    
for line in lines2[: min(num_samples, len(lines2) - 1)]:
    target_text= line.split("\t")
    target_texts.append(target_text)   

In [59]:
input_texts[500]



['" This is my third course and I bought this concept a long time ago.']

In [60]:
target_texts[500]

['" Det här är min tredje kurs och jag har köpt det hela för länge sedan.']

In [61]:
TRAIN_PERCENTAGE = 0.8
train_val = int(TRAIN_PERCENTAGE * num_samples)
train_val

800

In [62]:
text_pairs = list(zip(input_texts, target_texts))
random.shuffle(text_pairs)
input_texts, target_texts = zip(*text_pairs)
input_texts, target_texts = list(input_texts), list(target_texts)


In [63]:
train_input_texts = input_texts[:train_val]
train_target_texts = target_texts[:train_val]

val_input_texts = input_texts[train_val:]
val_target_texts = target_texts[train_val:]

In [64]:
spm.SentencePieceTrainer.train('--input=input_texts.txt --model_prefix=m --vocab_size=2000 --model_type=bpe')

# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

# encode: text => id
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

# decode: id => text
print(sp.decode_pieces(['▁This', '▁is', '▁a', '▁t', 'est']))
print(sp.decode_ids([664, 63, 4, 3, 133]))

['▁This', '▁is', '▁a', '▁t', 'est']
[664, 63, 4, 3, 133]
This is a test
This is a test


sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=input_texts.txt --model_prefix=m --vocab_size=2000 --model_type=bpe
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: input_texts.txt
  input_format: 
  model_prefix: m
  model_type: BPE
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_p